In [46]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib

from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import RationalQuadratic, Matern
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from sklearn.model_selection import GridSearchCV

In [47]:
# #PATH
path_info = "F:/178/Tugas Akhir/dataset/annotations/"
path_to_fitur = "F:/178/Tugas Akhir/dataset/coba fitur/"
path_to_file = "F:/178/Tugas Akhir/dataset/clips_45sec_wav/"
path_to_experiment ="F:/178/Tugas Akhir/dataset/experiment/"
path_to_model = "F:/178/Tugas Akhir/dataset/model/"
path_to_predict = "F:/178/Tugas Akhir/dataset/predict/"

# path_info = "C:/Users/USER/Downloads/Tugas Akhir/dataset/annotations/"
# path_to_fitur = "C:/Users/USER/Downloads/Tugas Akhir/dataset/coba fitur/"
# path_to_file = "C:/Users/USER/Downloads/Tugas Akhir/dataset/clips_45sec_wav/"
# path_to_experiment ="C:/Users/USER/Downloads/Tugas Akhir/dataset/experiment/"
# path_to_model = "C:/Users/USER/Downloads/Tugas Akhir/dataset/model/"
# path_to_predict = "C:/Users/USER/Downloads/Tugas Akhir/dataset/predict/"

In [48]:
def load(fitur):
    data = pd.read_csv(path_to_experiment + fitur+".csv",header=None)
    ground = pd.read_csv(path_info + "static_annotations.csv")

    arousal_mean = ground['mean_arousal']
    valence_mean = ground['mean_valence']
    
    return data,arousal_mean,valence_mean

def split(data,value):
    train_data, test_data, train_values, test_values = train_test_split(data,value,test_size=0.3,random_state=1000)
    
    return train_data, test_data, train_values, test_values
    
def predict(data,values,test,true,clf):
    trained = clf.fit(data,values)
    pred = clf.predict(test)
    df = pd.DataFrame(data=[pred,true])
    
    return pred,trained,df

def score(pred,true):
    r2 = r2_score(true, pred)
    
    return r2

# def gridsearch(data,true):
#     model = GPR()
# #     1.0 * RationalQuadratic(length_scale=1.0, alpha=0.1)
#     Cs = [1,1e1,1e2,1e3,1e4,1e5]
#     gammas = [1,1e-1,1e-2,1e-3,1e-4,1e-5]
#     parameters = {'C':Cs,'gamma':gammas}

#     clf = GridSearchCV(model, parameters, cv=10, scoring='r2')
#     clf.fit(data,true)

#     return clf.best_params_ , clf.best_score_

def regresion(clf,data,values):
    scores = cross_val_score(clf,data,values,scoring='r2',cv=10)
    
    return scores

def save_model(model,case,types):
    path = path_to_model+types+case+'norm_svr.sav' # save the model
    joblib.dump(model, path)

def save_predict(df1,df2,case):
    path = path_to_predict+case+"_norm_svr.xlsx" # save the model
    arousal = df1.T
    valence = df2.T
    merged = arousal.merge(valence, how='outer', left_index=True, right_index=True)
    merged.to_excel(path,header=None,index=None)


# CASE 1

In [49]:
datafitur = "case1_v2" #CASE 1

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,arousal_mean) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,valence_mean) #split valence
print(train_data_a.shape)

(744, 26)
(520, 26)


GRIDSEARCHCV

In [ ]:
# param_a, score_a = gridsearch(data,arousal_mean) #Gridsearch Arousal
# param_v, score_v = gridsearch(data,valence_mean) #Grinsearch Valence

In [10]:
# print("parameter Arousal:\t",param_a)
# print("parameter Valence:\t",param_v)

INITIAL MODEL

In [50]:
kernel = RationalQuadratic()

svr_case1_a = GPR(1.0 * kernel) #init Arousal
svr_case1_v = GPR(1.0 * kernel) #init Valence

TRAIN

In [51]:
# #Predict Arousal
a_a,train_a,df1_a = predict(train_data_a, train_values_a, test_data_a, test_values_a, svr_case1_a)
# #Predict Valence
a_v,train_v,df1_v = predict(train_data_v, train_values_v, test_data_v, test_values_v, svr_case1_v)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

AROUSAL

In [52]:
df1_a = df1_a.round(2)

display(df1_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,4.91,5.27,6.33,3.02,3.54,4.2,3.34,5.7,3.32,5.32,...,6.13,5.01,4.99,4.77,5.02,4.86,5.61,6.13,3.92,2.86
1,6.20,5.90,6.40,3.10,3.50,3.6,3.00,5.8,3.60,6.20,...,7.00,5.00,3.70,5.80,4.70,4.90,4.80,6.30,3.30,2.20


R2 Score Arousal:	0.55


VALENCE

In [53]:
df1_v = df1_v.round(2)

display(df1_v.head())
print("R2 Score Valence: %0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,5.48,4.97,5.14,4.19,4.24,4.68,4.39,5.22,4.05,5.35,...,4.94,5.22,5.49,5.17,5.55,5.19,4.82,5.35,4.61,3.93
1,2.70,6.40,6.00,2.70,5.00,3.20,4.60,4.30,4.10,6.70,...,6.60,4.80,2.50,5.60,3.80,4.40,6.50,6.50,4.00,3.50


R2 Score Valence: 0.04


REGRESSION

In [54]:
scores_a = regresion(svr_case1_a,data,arousal_mean) #Arousal Regression
scores_v = regresion(svr_case1_v,data,valence_mean) #Arousal Regression

AROUSAL

In [55]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.22112657  0.41117772  0.46662236  0.41666917  0.55507902  0.39328927
  0.52356987  0.44088086  0.60629488  0.52883298]
R2 Score Valence:	0.46 (+/- 0.10)


VALENCE

In [56]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [-0.53313724 -0.15806705  0.04104678 -0.24919868  0.15343497 -0.01214436
  0.07441263  0.17814109  0.18690935 -0.12680165]
R2 Score Valence:	-0.04 (+/- 0.22)


In [14]:
# save_predict(df1_a.round(2),df1_v.round(2),datafitur)
# save_model(train_a,datafitur,'arousal/') #save model Arousal
# save_model(train_v,datafitur,'valence/') #save model Valence

# CASE 2

In [57]:
datafitur = "case2" #CASE 2

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,arousal_mean) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,valence_mean) #split valence

(744, 34)


GRIDSEARCH

In [16]:
param_a, score_a = gridsearch(data,arousal_mean) #Gridsearch Arousal
param_v, score_v = gridsearch(data,valence_mean) #Grinsearch Valence

C:\Users\USER\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\USER\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [17]:
print("parameter Arousal:\t",param_a)
print("parameter Valence:\t",param_v)

parameter Arousal:	 {'C': 10000.0, 'gamma': 0.001}
parameter Valence:	 {'C': 100.0, 'gamma': 0.01}


INITIAL MODEL

In [58]:
kernel = RationalQuadratic()

svr_case2_a = GPR(1.0 * kernel) #init Arousal
svr_case2_v = GPR(1.0 * kernel) #init Valence


TRAIN

In [59]:
# #Predict Arousal
a_a,train_a,df2_a = predict(train_data_a, train_values_a, test_data_a, test_values_a, svr_case2_a)
# #Predict Valence
a_v,train_v,df2_v = predict(train_data_v, train_values_v, test_data_v, test_values_v, svr_case2_v)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

AROUSAL

In [60]:
df2_a = df2_a.round(2)

display(df2_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,5.09,5.63,6.56,3.54,3.35,4.43,3.24,6.09,3.32,5.25,...,6.34,5.19,4.9,4.83,5.16,4.71,5.5,6.38,4.08,2.88
1,6.20,5.90,6.40,3.10,3.50,3.60,3.00,5.80,3.60,6.20,...,7.00,5.00,3.7,5.80,4.70,4.90,4.8,6.30,3.30,2.20


R2 Score Arousal:	0.65


VALENCE

In [61]:
df2_v = df2_v.round(2)

display(df2_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,5.29,5.52,5.36,4.47,4.21,5.02,4.5,5.64,4.02,5.4,...,5.19,5.69,4.92,5.07,5.31,5.08,5.11,5.34,4.64,3.95
1,2.70,6.40,6.00,2.70,5.00,3.20,4.6,4.30,4.10,6.7,...,6.60,4.80,2.50,5.60,3.80,4.40,6.50,6.50,4.00,3.50


R2 Score Valence:	0.25


REGRESSION

In [62]:
scores_a = regresion(svr_case2_a,data,arousal_mean) #Arousal Regression
scores_v = regresion(svr_case2_v,data,valence_mean) #Arousal Regression

AROUSAL

In [63]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.41447484  0.48083023  0.51662143  0.40250185  0.69161554  0.52998385
  0.49227013  0.52294162  0.63396752  0.56527867]
R2 Score Valence:	0.53 (+/- 0.08)


VALENCE

In [64]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [-0.07355724 -0.00754699  0.00553093 -0.22261715  0.3801838   0.0918265
  0.15761349  0.22284385  0.32894446  0.04561479]
R2 Score Valence:	0.09 (+/- 0.18)


In [25]:
# save_predict(df2_a.round(2),df2_v.round(2),datafitur)
# save_model(train_a,datafitur,'arousal/') #save model Arousal
# save_model(train_v,datafitur,'valence/') #save model Valence

# CASE 3

In [65]:
datafitur =  "case3" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,arousal_mean) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,valence_mean) #split valence

(744, 37)


GRIDSEARCH

In [27]:
param_a, score_a = gridsearch(data,arousal_mean) #Gridsearch Arousal
param_v, score_v = gridsearch(data,valence_mean) #Grinsearch Valence

C:\Users\USER\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\USER\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [28]:
print("parameter Arousal:\t",param_a)
print("parameter Valence:\t",param_v)

parameter Arousal:	 {'C': 10000.0, 'gamma': 0.001}
parameter Valence:	 {'C': 10.0, 'gamma': 0.01}


INITAL MODEL

In [66]:
kernel = RationalQuadratic()

svr_case3_a = GPR(1.0 * kernel) #init Arousal
svr_case3_v = GPR(1.0 * kernel) #init Valence

TRAIN

In [67]:
# #Predict Arousal
a_a,train_a,df3_a = predict(train_data_a, train_values_a, test_data_a, test_values_a, svr_case3_a)
# #Predict Valence
a_v,train_v,df3_v = predict(train_data_v, train_values_v, test_data_v, test_values_v, svr_case3_v)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

AROUSAL

In [68]:
df3_a = df3_a.round(2)

display(df3_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,5.09,5.65,6.63,3.55,3.35,4.43,4.55,5.87,3.32,5.27,...,6.45,5.15,4.9,4.83,5.13,4.73,5.5,6.32,4.09,2.88
1,6.20,5.90,6.40,3.10,3.50,3.60,3.00,5.80,3.60,6.20,...,7.00,5.00,3.7,5.80,4.70,4.90,4.8,6.30,3.30,2.20


R2 Score Arousal:	0.63


VALENCE

In [69]:
df3_v = df3_v.round(2)

display(df3_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,5.31,5.5,5.54,4.44,4.16,5.02,4.58,5.76,3.95,5.45,...,5.41,5.79,5.01,5.05,5.37,5.05,5.18,5.09,4.64,3.89
1,2.70,6.4,6.00,2.70,5.00,3.20,4.60,4.30,4.10,6.70,...,6.60,4.80,2.50,5.60,3.80,4.40,6.50,6.50,4.00,3.50


R2 Score Valence:	0.27


REGRESSION

In [70]:
scores_a = regresion(svr_case3_a,data,arousal_mean) #Arousal Regression
scores_v = regresion(svr_case3_v,data,valence_mean) #Arousal Regression

AROUSAL

In [71]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.41561801  0.47306955  0.50804924  0.40015974  0.69175777  0.53858189
  0.48453056  0.56387605  0.60103361  0.5733748 ]
R2 Score Valence:	0.53 (+/- 0.08)


VALENCE

In [72]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ -4.47113919e-02   1.14126241e-04   2.78588228e-02  -1.85809267e-01
   3.55205898e-01   9.84740554e-02   1.24233541e-01   1.26166206e-01
   2.66894950e-01   1.22357940e-01]
R2 Score Valence:	0.09 (+/- 0.15)


In [36]:
# save_predict(df3_a.round(2),df3_v.round(2),datafitur)
# save_model(train_a,datafitur,'arousal/') #save model Arousal
# save_model(train_v,datafitur,'valence/') #save model Valence

# CASE 4

In [38]:
datafitur =  "case4" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,arousal_mean) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,valence_mean) #split valence

(744, 85)


GRIDSEAERCH

In [38]:
param_a, score_a = gridsearch(data,arousal_mean) #Gridsearch Arousal
param_v, score_v = gridsearch(data,valence_mean) #Grinsearch Valence

C:\Users\USER\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\USER\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [39]:
print("parameter Arousal:\t",param_a)
print("parameter Valence:\t",param_v)

parameter Arousal:	 {'C': 10000.0, 'gamma': 0.0001}
parameter Valence:	 {'C': 100.0, 'gamma': 0.001}


INITIAL MODEL

In [39]:
kernel = RationalQuadratic()

svr_case4_a = GPR(1.0 * kernel) #init Arousal
svr_case4_v = GPR(1.0 * kernel) #init Valence

TRAIN

In [40]:
# #Predict Arousal
a_a,train_a,df4_a = predict(train_data_a, train_values_a, test_data_a, test_values_a, svr_case4_a)
# #Predict Valence
a_v,train_v,df4_v = predict(train_data_v, train_values_v, test_data_v, test_values_v, svr_case4_v)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

AROUSAL

In [41]:
df4_a = df4_a.round(2)

display(df4_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,4.79,5.3,6.59,3.52,3.72,4.53,4.31,5.87,3.7,5.06,...,6.6,5.48,4.35,3.79,4.5,5.18,5.67,6.37,4.32,3.71
1,6.20,5.9,6.40,3.10,3.50,3.60,3.00,5.80,3.6,6.20,...,7.0,5.00,3.70,5.80,4.7,4.90,4.80,6.30,3.30,2.20


R2 Score Arousal:	0.55


VALENCE

In [42]:
df4_v = df4_v.round(2)

display(df4_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,4.44,5.64,5.6,4.91,4.81,4.92,4.11,5.2,4.65,5.1,...,5.48,5.65,4.73,4.28,4.71,5.25,5.02,4.77,4.78,4.28
1,2.70,6.40,6.0,2.70,5.00,3.20,4.60,4.3,4.10,6.7,...,6.60,4.80,2.50,5.60,3.80,4.40,6.50,6.50,4.00,3.50


R2 Score Valence:	0.23


REGRESSION

In [43]:
scores_a = regresion(svr_case4_a,data,arousal_mean) #Arousal Regression
scores_v = regresion(svr_case4_v,data,valence_mean) #Arousal Regression

AROUSAL

In [44]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.29619975  0.47475163  0.32499577  0.35566488  0.62310969  0.35876695
  0.43934603  0.40385719  0.58553436  0.62803753]
R2 Score Valence:	0.45 (+/- 0.12)


VALENCE

In [45]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [-0.05257229  0.01248185 -0.02319874 -0.09793413  0.27305424  0.06888972
  0.07502928  0.01138074  0.27088192  0.2520797 ]
R2 Score Valence:	0.08 (+/- 0.13)


In [47]:
# save_predict(df4_a.round(2),df4_v.round(2),datafitur)
# save_model(train_a,datafitur,'arousal/') #save model Arousal
# save_model(train_v,datafitur,'valence/') #save model Valence

In [ ]:
# # Load from file
# with open(arousal_model, 'rb') as file_arousal:
#     arousal_model = pickle.load(file_arousal)
# # # Load from file
# # with open(valence_model, 'rb') as file_valence:
# #     valence_model = pickle.load(file_valence)

# a = arousal_model.predict(test_data_a)